In [ ]:
# Install nibabel library, if not done yet (for reading gii files)
#!pip install nibabel
# Install openpyxl, if not done yet (for saving Excel file)
#!pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
import nibabel as nb
import os

In [2]:
#Data location
#noninjured.L.ab
baseFolder  = 'C:\\RaghusWork\\Data\\GarciaPNAS2018_example\\noninjured.L.ab'
youngerFile = 'YAS.LLR.surf.gii'
olderRegFile = 'configincaltrelaxforward.anat.LLR.reg.surf.gii'

In [3]:
#Read data
img = nb.load(os.path.join(baseFolder, youngerFile))
younger_coords = img.agg_data('NIFTI_INTENT_POINTSET')
younger_triangles = img.agg_data('NIFTI_INTENT_TRIANGLE')

In [ ]:
img = nb.load(os.path.join(baseFolder, olderRegFile))
olderReg_coords = img.agg_data('NIFTI_INTENT_POINTSET')
olderReg_triangles = img.agg_data('NIFTI_INTENT_TRIANGLE')
print(np.array_equal(younger_triangles, olderReg_triangles))
print(np.array_equal(younger_coords,olderReg_coords))

In [ ]:
#compute growth vectors
growthVector = olderReg_coords - younger_coords

In [ ]:
#Converting to pandas dataframe
younger_coords = pd.DataFrame(younger_coords)
olderReg_coords = pd.DataFrame(olderReg_coords)
younger_triangles = pd.DataFrame( younger_triangles)
growthVector = pd.DataFrame( growthVector)

In [ ]:
# Co-efficient computation
a0 = []; a1 = []; a2 = [];
b0 = []; b1 = []; b2 = [];
c0 = []; c1 = []; c2 = [];
for index, edges in younger_triangles.iterrows():
    #if (index>2):
    #    break
    vertex0 = younger_coords.iloc[edges[0]].to_numpy()
    vertex1 = younger_coords.iloc[edges[1]].to_numpy()
    vertex2 = younger_coords.iloc[edges[2]].to_numpy()
    normal = np.cross(vertex1-vertex0, vertex2-vertex0)
    G0 = growthVector.iloc[edges[0]].to_numpy()
    G1 = growthVector.iloc[edges[1]].to_numpy()
    G2 = growthVector.iloc[edges[2]].to_numpy()
    n2 = np.dot(normal,normal)
    a = np.cross((G1[0]-G0[0])*normal,(vertex0-vertex2)) + np.cross((G2[0]-G0[0])*normal,(vertex1-vertex0))
    a = a/n2
    a0.append(a[0]); a1.append(a[1]); a2.append(a[2]);
    b = np.cross((G1[1]-G0[1])*normal,(vertex0-vertex2)) + np.cross((G2[1]-G0[1])*normal,(vertex1-vertex0))
    b = b/n2
    b0.append(b[0]); b1.append(b[1]); b2.append(b[2]);
    c = np.cross((G1[2]-G0[2])*normal,(vertex0-vertex2)) + np.cross((G2[2]-G0[2])*normal,(vertex1-vertex0))
    c = c/n2
    c0.append(c[0]); c1.append(c[1]); c2.append(c[2]);
    #print(edges[0],edges[1],edges[2])  
    #print(vertex0,vertex1,vertex2,normal,n2)
    #print(a,b,c)

In [ ]:
#Saving results to Excel file
with pd.ExcelWriter('noninjured_L_ab.xlsx', engine='openpyxl') as writer:
    younger_coords.to_excel(writer, sheet_name='Week27Toward31-vertices')
    olderReg_coords.to_excel(writer, sheet_name='Week31For27-vertices')
    younger_triangles.to_excel(writer, sheet_name='Week27Toward31-triangles')
    outDf = pd.DataFrame({'a0': a0, 'a1': a1, 'a2': a2, 'b0': b0, 'b1': b1, 'b2': b2, 'c0': c0, 'c1': c1, 'c2': c2})
    outDf.to_excel(writer, sheet_name='co-efficients') 

In [ ]:
#Display of growth vectors
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection="3d")
stride = 50
x = younger_coords[::stride,0]
y = younger_coords[::stride,1]
z = younger_coords[::stride,2]
u = growthVector[::stride,0]
v = growthVector[::stride,1]
w = growthVector[::stride,2]
ax.quiver(x, y, z, u, v, w, length=2., arrow_length_ratio = 2, zorder=3, color='black', normalize= True) #, width=0.007, headwidth=3., headlength=4.) 
plt.show()

In [ ]:
#ORIENTABILITY code

In [4]:
def check_orientability(vertices, triangles):
    def get_edges(tri):
        return [(tri[i], tri[(i+1)%3]) for i in range(3)]

    def reverse_edge(edge):
        return (edge[1], edge[0])

    def edge_orientation(tri, edge):
        edge_idx = tri.index(edge[0])
        next_idx = (edge_idx + 1) % 3
        return tri[next_idx] == edge[1]

    edges_dict = {}
    for idx, tri in enumerate(triangles):
        for edge in get_edges(tri):
            if edge not in edges_dict:
                edges_dict[edge] = [tri]
            else:
                edges_dict[edge].append(tri)

    for edge, triangles in edges_dict.items():
        if len(triangles) == 2:
            tri1, tri2 = triangles
            ori_tri1 = edge_orientation(tri1, edge)
            ori_tri2 = edge_orientation(tri2, edge)
            if ori_tri1 == ori_tri2:
                return False

    return True

In [5]:
# Example usage
vertices = [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0)]
triangles = [(0, 1, 2), (1, 2, 3)]
print(check_orientability(vertices, triangles))  # Output: False
triangles = [(0, 1, 2), (1, 3, 2)]
print(check_orientability(vertices, triangles))  # Output: True
print(check_orientability(younger_coords, younger_triangles))

False
True
True
